In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast  # For mixed precision

from model import CIFAR10Net
from albumentations_transform import AlbumentationsTransform

class CIFAR10Dataset(Dataset):
    def __init__(self, data, targets, transform=None, train=True):
        self.data = data
        self.targets = targets
        self.transform = transform
        self.train = train

    def __getitem__(self, idx):
        img = self.data[idx]
        target = self.targets[idx]

        if self.transform:
            img = self.transform(img, self.train)

        return img, target

    def __len__(self):
        return len(self.data)

def train_one_epoch(model, device, train_loader, optimizer, criterion, epoch, scaler):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Mixed precision: Use autocast for the forward pass
        with autocast():
            output = model(data)
            loss = criterion(output, target)

        # Backward pass with mixed precision scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()  # Update the scale for the next iteration

        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(f'Epoch: {epoch} Loss: {loss.item():.4f} Acc: {100*correct/processed:.2f}%')

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%\n')
    return accuracy

def main():
    SEED = 42
    BATCH_SIZE = 64
    EPOCHS = 300
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    torch.manual_seed(SEED)

    # Get transforms
    transform = AlbumentationsTransform()

    # Load CIFAR10
    trainset = datasets.CIFAR10(root='./data', train=True, download=True)
    testset = datasets.CIFAR10(root='./data', train=False, download=True)

    # Create datasets
    train_dataset = CIFAR10Dataset(trainset.data, trainset.targets, transform, train=True)
    test_dataset = CIFAR10Dataset(testset.data, testset.targets, transform, train=False)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Initialize model, criterion, optimizer
    model = CIFAR10Net().to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.15, momentum=0.95)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.05)  # StepLR scheduler

    # Mixed precision: Initialize the scaler
    scaler = GradScaler()

    # Training loop
    best_acc = 0
    for epoch in range(1, EPOCHS + 1):
        train_one_epoch(model, DEVICE, train_loader, optimizer, criterion, epoch, scaler)
        acc = test(model, DEVICE, test_loader, criterion)
        scheduler.step()  # Update the learning rate using StepLR scheduler

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), 'best_model.pth')

        # Stop training if accuracy reaches 85%
        if acc >= 85.0:
            print(f"Stopping early as accuracy reached {acc:.2f}% at epoch {epoch}.")
            break

if __name__ == '__main__':
    main()


Files already downloaded and verified
Files already downloaded and verified


<ipython-input-5-f3625c3c9d82>:108: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
  0%|          | 0/782 [00:00<?, ?it/s]<ipython-input-5-f3625c3c9d82>:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch: 1 Loss: 1.6182 Acc: 37.09%: 100%|██████████| 782/782 [00:25<00:00, 30.75it/s]



Test set: Average loss: 1.4828, Accuracy: 44.95%



Epoch: 2 Loss: 1.0413 Acc: 50.91%: 100%|██████████| 782/782 [00:25<00:00, 30.24it/s]



Test set: Average loss: 1.1030, Accuracy: 60.24%



...


Test set: Average loss: 0.4522, Accuracy: 84.80%



Epoch: 33 Loss: 0.4422 Acc: 82.51%: 100%|██████████| 782/782 [00:24<00:00, 31.92it/s]



Test set: Average loss: 0.4476, Accuracy: 84.98%



Epoch: 34 Loss: 0.6095 Acc: 82.94%: 100%|██████████| 782/782 [00:25<00:00, 31.28it/s]



Test set: Average loss: 0.4489, Accuracy: 85.19%

Stopping early as accuracy reached 85.19% at epoch 34.
